# Fit the Model

In [2]:
%%sh
drum fit --code-dir ./model \
--input data/10K_Lending_Club_Loans_utf-8.csv \
--output ./model \
--target-type binary \
--target is_bad \
--docker env 
--verbose
--logging-level debug

Files were overwritten: {'/opt/fit_output_dir/model-config.yaml', '/opt/fit_output_dir/feature_detail.yaml', '/opt/fit_output_dir/create_pipeline.py', '/opt/fit_output_dir/athena_creds.yaml', '/opt/fit_output_dir/README.md', '/opt/fit_output_dir/azure_creds.yaml', '/opt/fit_output_dir/create_data.py', '/opt/fit_output_dir/artifact.pkl', '/opt/fit_output_dir/custom_model.py', '/opt/fit_output_dir/schema.json', '/opt/fit_output_dir/custom.py'}
Success 🎉
Host DRUM version: drum 1.4.8
Container DRUM version: 
the input device is not a TTY



# Testing the model

## Performance Testing

In [3]:
%%sh
drum perf-test \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type binary \
--positive-class-label 1 \
--negative-class-label 0 \
--docker env \
--verbose

Detected perf-test mode
DRUM performance test
Model:      /Users/timothy.whittaker/Desktop/git/ford-mlops/model
Data:       /Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv
# Features: 35
>78Preparing test data...
Running drum using: [drum server --code-dir /Users/timothy.whittaker/Desktop/git/ford-mlops/model --address localhost:53375 --logging-level warning --show-perf --target-type binary --positive-class-label 1 --negative-class-label 0 --docker env]



Running test case with timeout: 180
Running test case: 550 bytes - 1 samples, 100 iterations
Running test case with timeout: 180
Running test case: 0.1MB - 190 samples, 50 iterations
Running test case with timeout: 180
Running test case: 10MB - 19051 samples, 5 iterations
Running test case with timeout: 180
Running test case: 50MB - 95256 samples, 1 iterations
Test is done stopping drum server

 size    sample   iters    min     avg     max    contai   conta   conta   total
           s                       

## Validation

In [17]:
%%sh
drum validation \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type binary \
--positive-class-label 1 \
--negative-class-label 0 \
--docker env > validation.log

docker: Error response from daemon: invalid mode: /opt/input.csv.
See 'docker run --help'.
2020-12-29 10:58:22,775 ERROR drum:  Error from docker process: 125
Traceback (most recent call last):
  File "/usr/local/bin/drum", line 6, in <module>
    main()
  File "/usr/local/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/usr/local/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/usr/local/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 559, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.run_pipeline(cleanup=False)
  File "/usr/local/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 261, in run_pipeline
    self._run_pipeline()
  File "/usr/local/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 251, in _run_pipeline
    self._dag.run_connected_pipeline(self._ml_engin

In [18]:
! tail -n 30 validation.log

the input device is not a TTY



Validation checks results
      Test case         Status                       Details                    
Null value imputation   FAILED   Null value imputation check performs check by  
                                 imputing each feature with NaN value. If check 
                                 fails for a feature, test dataset is saved in  
                                 /tmp/drum_validation_checks_. Make sure to     
                                 delete those folders if it takes too much      
                                 space.                                         
                                                                                
                                 Failed feature                          Dataset
                                 filename                                       
                                 Unnamed: 0       /tmp/drum_validation_checks_jl
                                 mb3e8g/null_value

## Batch Scoring

In [5]:
%%sh
drum score \
--code-dir ./model \
--target-type binary \
--positive-class-label pos_class_prediction \
--negative-class-label neg_class_prediction \
--input data/10K_LC_short.csv \
--logging-level info \
--docker env \
--output data/predictions.csv

Host DRUM version: drum 1.4.8
Container DRUM version: 
the input device is not a TTY



2021-01-04 11:03:28,052 INFO drum:  Building a docker image from directory /Users/timothy.whittaker/Desktop/git/ford-mlops/env...
2021-01-04 11:03:28,052 INFO drum:  This may take some time
2021-01-04 11:03:28,611 INFO drum:  Done building image!
2021-01-04 16:03:30,708 INFO drum:  >>> Start drum in the score mode
2021-01-04 16:03:30,709 INFO drum..Executor:  Start initializing pipeline
2021-01-04 16:03:30,710 INFO drum..Executor:  Engine type: Generic
2021-01-04 16:03:30,711 INFO drum..Executor:  Using python engine
2021-01-04 16:03:30,717 INFO .ComponentsDesc:  Handling step ... engine: Generic, comp-type: generic_predictor
2021-01-04 16:03:31,629 INFO root:  init call, code_dir -> /opt/model
2021-01-04 16:03:31,629 INFO root:  init call, kwargs -> {}
2021-01-04 16:03:31,630 INFO root:  Linux-4.19.76-linuxkit-x86_64-with-debian-10.7
2021-01-04 16:03:31,630 INFO root:  ['/opt/model', '/usr/local/lib/python3.7/site-packages/datarobot_drum/resource/components/Python/generic_predictor', 

In [6]:
!head -n 5 data/predictions.csv

neg_class_prediction,pos_class_prediction
0.9869025253126411,0.013097474687358834
0.9050606618481462,0.09493933815185383
0.9795625695452991,0.020437430454700877
0.9188525700336793,0.08114742996632078


## Batch Scoring Unstructured Model

Returns Predictions and SHAP Values

In [7]:
%%sh
drum score \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type unstructured \
--content-type application/text \
--docker env \
--verbose --output data/predictions.json

Detected score mode
Detected /opt/model/custom.py .. trying to load hooks
Finished loading model, total used 100 iterations
 
 
Component: generic_predictor
Language:  Python
Output:
------------------------------------------------------------
Incoming content type params:  {'mimetype': 'application/text'}
Incoming data type:  <class 'bytes'>
Incoming query params:  {}
------------------------------------------------------------
Runtime:    0.2 sec
NR outputs: 0
 
docker command: [docker run --rm --interactive --user $(id -u):$(id -g)   -v /Users/timothy.whittaker/Desktop/git/ford-mlops/model:/opt/model -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv":/opt/input.csv -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/predictions.json":/opt/output.csv sha256:adbbe82490945f8bf4847f47751206900a8ecbdc672d653ea567023fff31d0cd drum score --code-dir /opt/model --input /opt/input.csv --target-type unstructured --content-type application/text --verbose --output /o

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [8]:
import json
import pprint
d = json.load(open("data/predictions.json", "rb"))
pprint.pprint(d["data"][0])

{'predictions': {'negative_class_label_prob': 0.9869025253126411,
                 'positive_class_label_prob': 0.013097474687358834},
 'shap_values': {'POS_CLASS_SHAP_addr_state': 0.0036035509567888145,
                 'POS_CLASS_SHAP_addr_state_name': 0.003491073098029413,
                 'POS_CLASS_SHAP_annual_inc': 0.11969366031099128,
                 'POS_CLASS_SHAP_delinq_2yrs': -0.011531092556902944,
                 'POS_CLASS_SHAP_dti': 0.01487584708518544,
                 'POS_CLASS_SHAP_earliest_cr_line day of week': -0.017397820412868293,
                 'POS_CLASS_SHAP_earliest_cr_line month of year': -0.1550894863647559,
                 'POS_CLASS_SHAP_emp_length': -0.11103693568966079,
                 'POS_CLASS_SHAP_emp_title': -0.02682982281083786,
                 'POS_CLASS_SHAP_funded_amnt': 0.026755169333029898,
                 'POS_CLASS_SHAP_grade': 0.006084105646094981,
                 'POS_CLASS_SHAP_home_ownership': -0.008111152884487917,
            

## Serving Model

In [23]:
import subprocess
import requests
import time

In [24]:
run_inference_server = [
    "drum", "server",
    "--code-dir","./model", 
    "--address", "0.0.0.0:6789", 
    "--show-perf",
    "--target-type", "binary",
    "--logging-level", "info",
    "--show-stacktrace",
    "--verbose", 
    "--docker", "env"
]
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [25]:
## snoozing before pinging the server to give it time to actually start
time.sleep(5)
print('check status')
requests.request("GET", "http://0.0.0.0:6789/").content

check status


b'{"message":"OK"}\n'

In [26]:
## shutdown
requests.request("POST", "http://0.0.0.0:6789/shutdown/").content

b'Server shutting down...'

In [27]:
inference_server.terminate()
inference_server.stdout.readlines()

[b'Detected REST server mode - this is an advanced option\n',
 b'Detected /opt/model/custom.py .. trying to load hooks\n',
 b'Finished loading model, total used 100 iterations\n',
 b'\x1b \x1b\n',
 b'\x1b \x1b\n',
 b'\x1b============================================================\x1b\n',
 b'\x1bComponent: prediction_server\x1b\n',
 b'\x1bLanguage:  Python\x1b\n',
 b'\x1bOutput:\x1b\n',
 b'\x1b------------------------------------------------------------\x1b\n',
 b' * Serving Flask app "datarobot_drum.drum.server" (lazy loading)\n',
 b' * Environment: production\n',
 b'   WARNING: This is a development server. Do not use it in a production deployment.\n',
 b'   Use a production WSGI server instead.\n',
 b' * Debug mode: off\n',
 b'docker command: [docker run --rm --interactive --user $(id -u):$(id -g)   -v /Users/timothy.whittaker/Desktop/git/ford-mlops/model:/opt/model -p 6789:6789 sha256:e45270890949cd2071f8822281b57de0b6a1ac394ad33a2122ec115b6b916981 drum server --code-dir /opt/model

## Push Artifacts and Environment to DataRobot

In [11]:
!python push.py --env-dir ./env --model-dir ./model --logging-level INFO --max-wait 600 --update-env True

Namespace(env_dir='./env', logging_level='INFO', max_wait=600, model_dir='./model', update_env=True)
__main__ - INFO - 2020-12-17 22:54:52,191 - Namespace(env_dir='./env', logging_level='INFO', max_wait=600, model_dir='./model', update_env=True)
__main__ - INFO - 2020-12-17 22:54:52,449 - updating environment
__main__ - INFO - 2020-12-17 22:54:52,451 - {'description': 'this is an environment to pair with lgbm model.', 'environmentVersionID': '5fda0b242d986b76e2d33bf8', 'id': '5fda0b23d33aebb30a492f74', 'name': 'lgbm execution environment', 'programmingLanguage': 'python'}
__main__ - INFO - 2020-12-17 22:54:52,786 - creating new execution environment version
__main__ - INFO - 2020-12-17 22:58:09,214 - create environment version complete
__main__ - INFO - 2020-12-17 22:58:09,214 - update env info yaml
__main__ - INFO - 2020-12-17 22:58:09,217 - load model config
__main__ - INFO - 2020-12-17 22:58:09,221 - {'description': None, 'environmentID': '5fda0b23d33aebb30a492f74', 'environmentVers

# Test Model in DataRobot

In [12]:
!python test.py --model-dir ./model --logging-level INFO --max-wait 600

Namespace(logging_level='INFO', max_wait=600, model_dir='./model')
__main__ - INFO - 2020-12-17 22:59:31,044 - Namespace(logging_level='INFO', max_wait=600, model_dir='./model')
__main__ - INFO - 2020-12-17 22:59:31,044 - loading model config
__main__ - INFO - 2020-12-17 22:59:31,052 - starting custom model test
__main__ - INFO - 2020-12-17 23:07:03,930 - test complete
__main__ - INFO - 2020-12-17 23:07:03,931 - failed 😱
__main__ - ERROR - 2020-12-17 23:07:03,932 - Test: error_check
__main__ - ERROR - 2020-12-17 23:07:03,933 - Status: failed
__main__ - ERROR - 2020-12-17 23:07:03,933 - Message: ERROR: Model transform hook failed to transform dataset: Execution failed on sql: SELECT * FROM cfds_athena_demo.us_zip_codes
Could not connect to the endpoint URL: "https://athena.ap-south-1.amazonaws.com/"
unable to rollback
__main__ - ERROR - 2020-12-17 23:07:03,933 - Test: null_value_imputation
__main__ - ERROR - 2020-12-17 23:07:03,933 - Status: aborted
__main__ - ERROR - 2020-12-17 23:07:0